# Datasets extraction and merging

Data source : [Open Data Gouv](https://datafoncier.cerema.fr/donnees/autres-donnees-foncieres/dvfplus-open-data?fbclid=IwAR0_9x5Ijq0Bl8GY3nb7kmsJrAlN-RULISA9k7amFE5J4WNmPxLB0yUocaw) (april 2020)

----
*In collaboration with :*

- [Khaled Larbi](https://github.com/khaledlarbi)
- [Wenceslas Sanchez](https://github.com/Orlogskapten)

## Extraction

In [1]:
import os
from py7zr import unpack_7zarchive
import shutil

In [ ]:
# data_path= "data/shp_csv/"

# all_dir= os.listdir(data_path)
# print(all_dir)

# all_dir_path= list(map(lambda x: data_path+x, all_dir))
# print(all_dir_path)

# unziped_path= []
# for folder_path in all_dir_path:
#     file_name= os.listdir(folder_path)[0] # because there is only one file per folder
#     unziped_path.append(folder_path+"/"+file_name)
# print(unziped_path)

# # # Extraction
# for unziped in unziped_path:
#     shutil.unpack_archive(unziped, r"data/unziped")
    
# # Lighting our folders (around 15go of data)
# data_path= "data/unziped/"
# all_dir= os.listdir(data_path)

# for chiant in all_dir:
#     full_file_unzipped= os.listdir(data_path+chiant)
#     # I hate double for loop
#     for item in full_file_unzipped:
#         if item.endswith((".cpg", ".dbf", ".prj", ".cpg", ".prj")):
#             os.remove(os.path.join(data_path+chiant, item))

In [ ]:
def actions(data_path= r"data/shp_csv/", unziped_location= r"data/unziped"):
#     data_path= "data/shp_csv/"

    all_dir= os.listdir(data_path)
    print(all_dir)

    all_dir_path= list(map(lambda x: data_path+x, all_dir))
    print(all_dir_path)

    unziped_path= []
    for folder_path in all_dir_path:
        file_name= os.listdir(folder_path)[0] # because there is only one file per folder
        unziped_path.append(folder_path+"/"+file_name)
    print(unziped_path)
    
    try:
        shutil.register_unpack_format('7zip', ['.7z'], unpack_7zarchive)
    except:
        pass
    
    # Extraction
    for unziped in unziped_path:
        shutil.unpack_archive(unziped, unziped_location)

    # Lighting our folders (around 15go of data)
    all_dir= os.listdir(unziped_location)

#     for chiant in all_dir:
#         full_file_unzipped= os.listdir(unziped_location+"/"+chiant)
#         # I hate double for loop
#         for item in full_file_unzipped:
#             if item.endswith((".cpg", ".dbf", ".prj", ".cpg", ".prj")):
#                 os.remove(os.path.join(unziped_location+chiant, item))

actions()

In [ ]:
# import geopandas as gpd
# shapefile= gpd.read_file(r"C:\Users\wenceslas\Documents\jupyter_notebook\ensae\dvf\data\unziped\r01\r01_mutation_geomparmut.shp") 
# print(shapefile)
# shapefile.plot()

## Merging

In [2]:
import pandas as pd
import geopandas as gpd

In [8]:
unziped_dir= "data/unziped_csv"
all_dir_path= os.listdir(unziped_dir)

old_name= "mutation.csv"
new_name= "amutation.csv"
df_merged= pd.DataFrame()
row_num= 0

try:
    del df # aware
except: 
    pass

i= 0
# TOO MUCH FOR BOUND ! BE AWARE !
for folder in all_dir_path:
    # I change the csv file name to make it appear in first in the next part of the script
    try:
        old_path= os.path.join(unziped_dir, folder, folder+"_"+old_name).replace("\\", "/")
        new_path= os.path.join(unziped_dir, folder, folder+"_"+new_name).replace("\\", "/")
        os.rename(old_path, new_path)
    except OSError:
        pass
    
    all_files_in_folder= os.listdir(unziped_dir+"/" + folder)
#     if i <= 1:
    #
    for item in all_files_in_folder:

        # Find the csv file (the first one logically)
        if item.endswith((".csv")):
            csv_path= os.path.join(unziped_dir, folder, item).replace("\\", "/")
            print(csv_path)
            # I want to create df once, and then to merge all csv with df
            if "df" in globals() or "df" in locals():
                continue
            else:
                df= pd.read_csv(csv_path)
            print(df.shape)
            row_num += df.shape[0]

        elif item.endswith((".shp")):
            if "mutation_geo" in item:
                shape_path= os.path.join(unziped_dir, folder, item).replace("\\", "/")
                shapefile= gpd.read_file(shape_path)
                df_geo= pd.DataFrame(shapefile)
                del shapefile

                df_geo.columns= map(str.lower, df_geo.columns)
                # Merging step
                df= df.merge(df_geo, how= "left", on= "idmutation")
                del df_geo
                print(df.shape)
                

#             df= df.rename(columns= {"mutgeomlocmut": "geometry_x", "mutgeopar": "geometry_y"
#                , "mutgeoparmut": "geometry"})



    df.to_csv("data/final_csv/dvf_part_{}.csv".format(i), index= False)
    del df
    print("Saved")
    i += 1
#     df_concat= pd.concat([df_merged, df]).reset_index(drop= True)
#     df_merged= df_concat.copy()
#     print("Merged shape : {}".format(df_merged.shape))
#     del df, df_concat

    #    
        
#         i += 1
#     else:
#         break
# del df_concat
# assert row_num == df_merged.shape[0]


data/unziped_csv/r01/r01_amutation.csv
(17449, 61)
(17449, 62)
(17449, 63)
(17449, 64)
Saved
data/unziped_csv/r02/r02_amutation.csv
(14339, 61)
(14339, 62)
(14339, 63)
(14339, 64)
Saved
data/unziped_csv/r03/r03_amutation.csv
(9446, 61)
(9446, 62)
(9446, 63)
(9446, 64)
Saved
data/unziped_csv/r04/r04_amutation.csv
(35468, 61)
(35468, 62)
(35468, 63)
(35468, 64)
Saved
data/unziped_csv/r11/r11_amutation.csv
(1218527, 61)
(1218527, 62)
(1218527, 63)
(1218527, 64)
Saved
data/unziped_csv/r24/r24_amutation.csv
(307041, 61)
(307041, 62)
(307041, 63)
(307041, 64)
Saved
data/unziped_csv/r27/r27_amutation.csv
(324196, 61)
(324196, 62)
(324196, 63)
(324196, 64)
Saved
data/unziped_csv/r28/r28_amutation.csv
(354506, 61)
(354506, 62)
(354506, 63)
(354506, 64)
Saved
data/unziped_csv/r32/r32_amutation.csv
(569592, 61)
(569592, 62)
(569592, 63)
(569592, 64)
Saved
data/unziped_csv/r44/r44_amutation.csv
(297653, 61)
(297653, 62)
(297653, 63)
(297653, 64)
Saved
data/unziped_csv/r52/r52_amutation.csv
(533909

In [ ]:
final_csv= "data/final_csv"
all_dir_path= os.listdir(final_csv)
df= pd.DataFrame()
row_num= 0

i= 0
    
for csv_file in all_dir_path:
    shape_path= os.path.join(final_csv, csv_file).replace("\\", "/")
    print(csv_file)
#     if i <= 1:
    #
    df_to_merge= pd.read_csv(shape_path)
    row_num += df_to_merge.shape[0]

    df= pd.concat([df, df_to_merge]).reset_index(drop= True)
    del df_to_merge
    #

#     i += 1
    
assert row_num == df.shape[0]

In [10]:
df.to_csv("data/final_csv/dvf_entier.csv", index= False)
del df

,idmutation,idmutinvar,idopendata,idnatmut,codservch,refdoc,datemut,anneemut,moismut,coddep,...,smai1pp,smai2pp,smai3pp,smai4pp,smai5pp,codtypbien,libtypbien,geometry_x,geometry_y,geometry
0,7385547,31f7173bdb15e053660b96aa3c821542,31f7173bdb15e053660b96aa3c821542,1,NaN,NaN,2014-01-15,2014,1,971,...,0,0,0,0,0,121,UN APPARTEMENT,POINT (651415.9571617388 1783952.099946088),"POLYGON ((651400.1520208376 1783960.804038909,...",NaN
1,7289417,2973e645aa4b07a8ad78eba09cef4bb0,2973e645aa4b07a8ad78eba09cef4bb0,1,NaN,NaN,2016-02-15,2016,2,971,...,0,0,0,92,0,152,BATI MIXTE - LOGEMENT/ACTIVITE,POINT (681982.3379059634 1796729.128741806),"POLYGON ((681808.0969904528 1796736.92158914, ...","POLYGON ((681808.0969904528 1796736.92158914, ..."
2,7387630,dfdd91a0ea82ff63566cef001cbfdfad,dfdd91a0ea82ff63566cef001cbfdfad,1,NaN,NaN,2014-10-07,2014,10,971,...,0,0,0,0,0,2313,TERRAIN DE TYPE TERRE ET PRE,NaN,NaN,NaN
3,7289289,1f66bd22b3c99f5658ffc0a4731e8189,1f66bd22b3c99f5658ffc0a4731e8189,2,NaN,NaN,2016-09-28,2016,9,971,...,0,0,0,0,0,101,BATI - INDETERMINE : Vefa sans descriptif,NaN,NaN,NaN
4,7237051,f976ca2e93f9b1b9e83d48f4584200e9,f976ca2e93f9b1b9e83d48f4584200e9,1,NaN,NaN,2017-07-03,2017,7,971,...,0,0,0,0,0,121,UN APPARTEMENT,POINT (490166.5498510478 1997190.031305496),"POLYGON ((490166.2289086806 1997192.465448873,...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31783,7294098,c6c90b25d5b4cca5f2635181751e3b3c,c6c90b25d5b4cca5f2635181751e3b3c,1,NaN,NaN,2016-07-29,2016,7,972,...,0,0,0,0,0,21,TERRAIN DE TYPE TAB,NaN,"POLYGON ((719519.2891137758 1604597.79471469, ...","POLYGON ((719519.2891137758 1604597.79471469, ..."
31784,7176534,10cff935bc876dca25253acb910628f0,10cff935bc876dca25253acb910628f0,1,NaN,NaN,2018-07-23,2018,7,972,...,0,0,0,0,0,121,UN APPARTEMENT,POINT (705155.7607127766 1615990.527437198),"POLYGON ((705187.4170303441 1615994.848901571,...",NaN
31785,7239531,dae4df8da8a720678990a8153e6aae74,dae4df8da8a720678990a8153e6aae74,1,NaN,NaN,2017-12-15,2017,12,972,...,0,0,0,0,0,122,DEUX APPARTEMENTS,POINT (708745.3063396518 1607669.380934893),"POLYGON ((708728.1503777069 1607669.932495049,...","POLYGON ((708728.1503777069 1607669.932495049,..."
31786,7292370,2554e5ed06695f990e69cb16465b491c,2554e5ed06695f990e69cb16465b491c,1,NaN,NaN,2016-04-19,2016,4,972,...,0,0,51,0,0,111,UNE MAISON,POINT (707414.465020251 1615418.224367614),"POLYGON ((707418.3824360785 1615422.102896009,...","POLYGON ((707418.3824360785 1615422.102896009,..."


In [ ]:
df["idmutation"].nunique()

In [ ]:
shapefile= gpd.read_file(r"C:\Users\wenceslas\Documents\jupyter_notebook\ensae\dvf\data\unziped_csv\r01\r01_mutation_geomparmut.shp") 

df_geo= pd.DataFrame(shapefile)
print(df_geo.nunique())
print(df_geo.shape)

In [ ]:
test= df_geo.duplicated(["IDMUTATION"], keep= False)
# test= df_geo[test].sort_values(["IDMUTATION"#]).reset_index(drop= True)
print(test.head(10))
# df_geo
test["geometry"].iloc[0]

In [ ]:
test["geometry"].iloc[1]

In [ ]:
shapefile= gpd.read_file(r"C:\Users\wenceslas\Documents\jupyter_notebook\ensae\dvf\data\unziped_csv\r01\r01_mutation_geomlocmut.shp") 
# print(shapefile)
# shapefile.plot()
df_geo= pd.DataFrame(shapefile)
df_geo